In [ ]:
import os, numpy as np, pandas as pd, lightgbm as lgb, pyarrow as pa, pyarrow.parquet as pq
from sklearn.model_selection import TimeSeriesSplit
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import optuna
import kaggle_evaluation.default_inference_server
import warnings
import os
warnings.filterwarnings('ignore')

print("Available input folders:", os.listdir("/kaggle/input"))
print("Contents of my dataset folder:")
for root, dirs, files in os.walk("/kaggle/input"):
    for f in files:
        if "lgb" in f or "metadata" in f:
            print(os.path.join(root, f))

# CONFIG 
TARGET_COL = "market_forward_excess_returns"

# Paths for SAVING (Training Notebook - /kaggle/working/)
MODEL_SAVE_PATH_MOM = "/kaggle/working/lgb_model_mom.txt"
MODEL_SAVE_PATH_REV = "/kaggle/working/lgb_model_rev.txt"
MODEL_SAVE_PATH_VOL = "/kaggle/working/lgb_model_vol.txt"
METADATA_SAVE_PATH = "/kaggle/working/model_metadata.npy"

# Paths for LOADING (Submission Notebook - /kaggle/input/<dataset-name>/)
MODEL_LOAD_DIR =  "/kaggle/input/hull-lgb-trained-models"
MODEL_LOAD_PATH_MOM = f"{MODEL_LOAD_DIR}/lgb_model_mom.txt"
MODEL_LOAD_PATH_REV = f"{MODEL_LOAD_DIR}/lgb_model_rev.txt"
MODEL_LOAD_PATH_VOL = f"{MODEL_LOAD_DIR}/lgb_model_vol.txt"
METADATA_LOAD_PATH = f"{MODEL_LOAD_DIR}/model_metadata.npy"

TARGET_VOL, _model_mom, _model_rev, _model_vol, _initialized, _metadata = None, None, None, None, False, None
MIN_INVESTMENT, MAX_INVESTMENT = 0, 2
USE_OPTUNA = False  # Set to True for hyperparameter tuning (slower but better)
N_CV_FOLDS = 10  # Increased from 5
OPTUNA_TRIALS = 30  # Number of hyperparameter trials (reduced for speed)


# CUSTOM EVALUATION METRIC 
class ParticipantVisibleError(Exception): pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    if not pd.api.types.is_numeric_dtype(submission['prediction']):
        raise ParticipantVisibleError('Predictions must be numeric')
    solution = solution.copy()
    solution['position'] = submission['prediction']
    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position {solution["position"].max()} exceeds max {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position {solution["position"].min()} below min {MIN_INVESTMENT}')
    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + \
                                   solution['position'] * solution['forward_returns']
    strat_excess = solution['strategy_returns'] - solution['risk_free_rate']
    strat_cum = (1 + strat_excess).prod()
    strat_mean = strat_cum ** (1 / len(solution)) - 1
    strat_std = solution['strategy_returns'].std()
    if strat_std == 0: raise ParticipantVisibleError('Division by zero, strategy std is zero')
    trading_days = 252
    sharpe = strat_mean / strat_std * np.sqrt(trading_days)
    strat_vol = float(strat_std * np.sqrt(trading_days) * 100)
    mkt_excess = solution['forward_returns'] - solution['risk_free_rate']
    mkt_cum = (1 + mkt_excess).prod()
    mkt_mean = mkt_cum ** (1 / len(solution)) - 1
    mkt_std = solution['forward_returns'].std()
    if mkt_std == 0: raise ParticipantVisibleError('Division by zero, market std is zero')
    mkt_vol = float(mkt_std * np.sqrt(trading_days) * 100)
    excess_vol = max(0, strat_vol / mkt_vol - 1.2) if mkt_vol > 0 else 0
    vol_penalty = 1 + excess_vol
    return_gap = max(0, (mkt_mean - strat_mean) * 100 * trading_days)
    return_penalty = 1 + (return_gap ** 2) / 100
    adj_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adj_sharpe), 1_000_000)


# ENHANCED FEATURE ENGINEERING 
def calculate_rsi(series, period=14):
    """Calculate Relative Strength Index"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(series, fast=12, slow=26, signal=9):
    """Calculate MACD indicators"""
    ema_fast = series.ewm(span=fast).mean()
    ema_slow = series.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

def feature_engineer(df, is_inference=False, feature_importance=None, top_n_features=None):
    df = df.sort_values("date_id").reset_index(drop=True).copy()
    
    # Identify base features
    base_feats = [c for c in df.columns 
                  if c not in ["date_id", "risk_free_rate", "forward_returns", TARGET_COL, "target"]]
    
    feature_dict = {}
    
    # For each base feature, create technical indicators
    for c in base_feats:
        # Basic lags (reduced from [1,2,5,10] to [1,5,10])
        for lag in [1, 5, 10]:
            feature_dict[f"{c}_lag_{lag}"] = df[c].shift(lag).values
        
        # Rolling statistics (more selective)
        for w in [5, 10, 20]:
            feature_dict[f"{c}_ma_{w}"] = df[c].shift(1).rolling(w).mean().values
            feature_dict[f"{c}_std_{w}"] = df[c].shift(1).rolling(w).std().values
        
        # Technical indicators
        feature_dict[f"{c}_rsi_14"] = calculate_rsi(df[c], 14).values
        
        macd, macd_sig, macd_hist = calculate_macd(df[c])
        feature_dict[f"{c}_macd"] = macd.values
        feature_dict[f"{c}_macd_signal"] = macd_sig.values
        feature_dict[f"{c}_macd_hist"] = macd_hist.values
        
        # Momentum indicators
        for period in [5, 10, 20]:
            feature_dict[f"{c}_mom_{period}"] = (df[c] / df[c].shift(period) - 1).values
        
        # Volatility
        for w in [5, 10, 20]:
            feature_dict[f"{c}_vol_{w}"] = df[c].shift(1).rolling(w).std().values
        
        # Rate of change
        feature_dict[f"{c}_roc_5"] = (df[c] - df[c].shift(5)) / df[c].shift(5).values
    
    # Combine all features
    feat_df = pd.DataFrame(feature_dict, index=df.index)
    df = pd.concat([df, feat_df], axis=1)
    
    # Handle inf and nan
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill()
    
    # Feature selection based on importance (if available)
    if feature_importance is not None and top_n_features is not None and not is_inference:
        feature_cols = [c for c in df.columns if c not in base_feats + 
                               ["date_id", "risk_free_rate", "forward_returns", TARGET_COL, "target"]]
        
        # Keep only top features
        top_features = feature_importance.nlargest(top_n_features).index.tolist()
        keep_cols = base_feats + ["date_id", "risk_free_rate", "forward_returns", TARGET_COL, "target"] + top_features
        df = df[[c for c in keep_cols if c in df.columns]]
    
    return df

def remove_correlated_features(X, threshold=0.95):
    """Remove highly correlated features"""
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"  Removing {len(to_drop)} highly correlated features (threshold={threshold})")
    return X.drop(columns=to_drop), to_drop

# HYPERPARAMETER OPTIMIZATION 
def objective(trial, X, y, features, tscv):
    """Optuna objective function for hyperparameter tuning"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'verbose': -1,
        'random_state': 42
    }
    
    cv_scores = []
    for tr_i, va_i in tscv.split(X):
        Xtr, Xva = X.iloc[tr_i], X.iloc[va_i]
        ytr, yva = y.iloc[tr_i], y.iloc[va_i]
        
        model = lgb.LGBMRegressor(**params)
        model.fit(Xtr[features], ytr, 
                  eval_set=[(Xva[features], yva)],
                  callbacks=[lgb.early_stopping(30, verbose=False)])
        
        pred = model.predict(Xva[features])
        # Use correlation as optimization target (better than MSE for this task)
        corr = np.corrcoef(pred, yva)[0, 1]
        cv_scores.append(corr)
    
    return np.mean(cv_scores)

# OPTIMIZE ENSEMBLE WEIGHTS 
def optimize_ensemble_weights(predictions_dict, y_true, forward_returns, risk_free_rate):
    """Find optimal ensemble weights via grid search"""
    best_sharpe = -np.inf
    best_weights = (0.5, 0.3, 0.2)  # Default fallback
    best_scale = 5  # Default fallback
    
    # Convert to numpy arrays
    pred_mom = np.array(predictions_dict['mom'])
    pred_rev = np.array(predictions_dict['rev'])
    pred_vol = np.array(predictions_dict['vol'])
    forward_returns = np.array(forward_returns)
    risk_free_rate = np.array(risk_free_rate)
    
    print("  Testing weight combinations...")
    tested = 0
    successful = 0
    
    # Grid search over weights and scaling factor
    for w1 in np.linspace(0.3, 0.7, 9):
        for w2 in np.linspace(0.1, 0.5, 9):
            w3 = 1 - w1 - w2
            if w3 < 0.1 or w3 > 0.6:
                continue
            
            for scale in [3, 4, 5, 6, 7, 8, 9, 10]:
                tested += 1
                try:
                    p = w1 * pred_mom + w2 * pred_rev + w3 * pred_vol
                    pred = np.clip(1 + np.tanh(p * scale), 0, 2)
                    
                    sol = pd.DataFrame({
                        'forward_returns': forward_returns,
                        'risk_free_rate': risk_free_rate,
                        'prediction': pred
                    })
                    
                    s = score(sol, sol, row_id_column_name="date_id")
                    successful += 1
                    
                    if s > best_sharpe:
                        best_sharpe = s
                        best_weights = (w1, w2, w3)
                        best_scale = scale
                except Exception as e:
                    continue
    
    print(f"  Tested {tested} combinations, {successful} successful")
    return best_weights, best_scale, best_sharpe

# TRAIN MODEL
def train_model():
    global TARGET_VOL
    print("\n" + "="*70)
    print("ENHANCED TRAINING PIPELINE - OPTIMIZATION MODE")
    print("="*70)
    
    print("\n" + "="*70)
    print("LOADING DATA")
    print("="*70)
    tr = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv").dropna()
    te = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv").dropna()
    print(f"Training samples: {len(tr)}")
    print(f"Test samples: {len(te)}")
    
    TARGET_VOL = tr["forward_returns"].std()
    print(f"Target volatility: {TARGET_VOL:.6f}")
    
    tr["target"] = tr[TARGET_COL].shift(-1)
    tr = tr.dropna(subset=["target"])
    
    print("\n" + "="*70)
    print("ENHANCED FEATURE ENGINEERING")
    print("="*70)
    tr = feature_engineer(tr)
    print(f"Total features created: {len(tr.columns)}")
    
    X = tr.drop(columns=["target", TARGET_COL, "date_id", "forward_returns", "risk_free_rate"], errors='ignore')
    y = tr["target"]
    
    # Remove highly correlated features
    X, dropped_corr = remove_correlated_features(X, threshold=0.95)
    print(f"Features after correlation removal: {len(X.columns)}")
    
    # Separate feature types
    mom_features = [c for c in X.columns if any(x in c for x in ['_lag_', '_ma_', '_mom_', '_roc_'])]
    vol_features = [c for c in X.columns if any(x in c for x in ['_std_', '_vol_', '_rsi_'])]
    rev_features = [c for c in X.columns if any(x in c for x in ['_macd'])]
    
    # Add remaining to reversal
    used = set(mom_features + vol_features + rev_features)
    rev_features += [c for c in X.columns if c not in used]
    
    print(f"Momentum features: {len(mom_features)}")
    print(f"Volatility features: {len(vol_features)}")
    print(f"Reversal features: {len(rev_features)}")
    
    # Cross-validation setup
    print("\n" + "="*70)
    print(f"TIME SERIES CROSS-VALIDATION ({N_CV_FOLDS} Folds)")
    print("="*70)
    
    tscv = TimeSeriesSplit(n_splits=N_CV_FOLDS)
    
    # Hyperparameter tuning with Optuna
    best_params_mom, best_params_rev, best_params_vol = None, None, None
    
    if USE_OPTUNA:
        print("\nOptimizing hyperparameters with Optuna...")
        print("(This may take several minutes...)")
        
        # Tune momentum model
        print("  Tuning Momentum model...")
        study_mom = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study_mom.optimize(lambda trial: objective(trial, X, y, mom_features, TimeSeriesSplit(n_splits=3)), 
                           n_trials=OPTUNA_TRIALS, show_progress_bar=False)
        best_params_mom = study_mom.best_params
        print(f"    Best correlation: {study_mom.best_value:.6f}")
        
        # Tune reversal model
        print("  Tuning Reversal model...")
        study_rev = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=43))
        study_rev.optimize(lambda trial: objective(trial, X, y, rev_features, TimeSeriesSplit(n_splits=3)), 
                           n_trials=OPTUNA_TRIALS, show_progress_bar=False)
        best_params_rev = study_rev.best_params
        print(f"    Best correlation: {study_rev.best_value:.6f}")
        
        # Tune volatility model
        print("  Tuning Volatility model...")
        study_vol = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=44))
        study_vol.optimize(lambda trial: objective(trial, X, y, vol_features, TimeSeriesSplit(n_splits=3)), 
                           n_trials=OPTUNA_TRIALS, show_progress_bar=False)
        best_params_vol = study_vol.best_params
        print(f"    Best correlation: {study_vol.best_value:.6f}")
    else:
        # Default improved parameters with better regularization
        best_params_mom = {
            'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 64, 
            'min_child_samples': 30, 'subsample': 0.8, 'colsample_bytree': 0.8,
            'reg_alpha': 0.1, 'reg_lambda': 0.1, 'max_depth': 6, 
            'verbose': -1, 'random_state': 42
        }
        best_params_rev = {
            'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 64,
            'min_child_samples': 30, 'subsample': 0.8, 'colsample_bytree': 0.8,
            'reg_alpha': 0.1, 'reg_lambda': 0.1, 'max_depth': 6,
            'verbose': -1, 'random_state': 43
        }
        best_params_vol = {
            'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 64,
            'min_child_samples': 30, 'subsample': 0.8, 'colsample_bytree': 0.8,
            'reg_alpha': 0.1, 'reg_lambda': 0.1, 'max_depth': 6,
            'verbose': -1, 'random_state': 44
        }
    
    # Cross-validation with optimized models
    print("\nRunning full cross-validation...")
    scores = []
    fold_predictions_all = []
    fold_actuals_all = []
    all_preds_dict = {'mom': [], 'rev': [], 'vol': []}
    all_forward_returns = []
    all_risk_free_rate = []
    
    for fold, (tr_i, va_i) in enumerate(tscv.split(X), 1):
        Xtr, Xva = X.iloc[tr_i], X.iloc[va_i]
        ytr, yva = y.iloc[tr_i], y.iloc[va_i]
        
        m1 = lgb.LGBMRegressor(**best_params_mom)
        m2 = lgb.LGBMRegressor(**best_params_rev)
        m3 = lgb.LGBMRegressor(**best_params_vol)
        
        m1.fit(Xtr[mom_features], ytr, eval_set=[(Xva[mom_features], yva)],
              callbacks=[lgb.early_stopping(30, verbose=False)])
        m2.fit(Xtr[rev_features], ytr, eval_set=[(Xva[rev_features], yva)],
              callbacks=[lgb.early_stopping(30, verbose=False)])
        m3.fit(Xtr[vol_features], ytr, eval_set=[(Xva[vol_features], yva)],
              callbacks=[lgb.early_stopping(30, verbose=False)])
        
        p1 = m1.predict(Xva[mom_features])
        p2 = m2.predict(Xva[rev_features])
        p3 = m3.predict(Xva[vol_features])
        
        all_preds_dict['mom'].extend(p1)
        all_preds_dict['rev'].extend(p2)
        all_preds_dict['vol'].extend(p3)
        all_forward_returns.extend(tr.iloc[va_i]['forward_returns'].values)
        all_risk_free_rate.extend(tr.iloc[va_i]['risk_free_rate'].values)
        
        # Use default weights for fold scoring
        p = 0.5 * p1 + 0.3 * p2 + 0.2 * p3
        pred = np.clip(1 + np.tanh(p * 5), 0, 2)
        
        fold_predictions_all.extend(pred)
        fold_actuals_all.extend(yva.values)
        
        sol = tr.iloc[va_i][["forward_returns", "risk_free_rate"]].copy()
        sol["prediction"] = pred
        
        try:
            s = score(sol, sol, row_id_column_name="date_id")
            scores.append(s)
            print(f"  Fold {fold:2d} - Adjusted Sharpe: {s:.6f}")
        except Exception as e:
            scores.append(0)
            print(f"  Fold {fold:2d} - Error: {e}")
    
    # Optimize ensemble weights
    print("\n" + "="*70)
    print("OPTIMIZING ENSEMBLE WEIGHTS")
    print("="*70)
    best_weights, best_scale, opt_sharpe = optimize_ensemble_weights(
        all_preds_dict, fold_actuals_all, all_forward_returns, all_risk_free_rate
    )
    print(f"Optimal weights: Mom={best_weights[0]:.3f}, Rev={best_weights[1]:.3f}, Vol={best_weights[2]:.3f}")
    print(f"Optimal scaling factor: {best_scale}")
    print(f"Optimized Sharpe: {opt_sharpe:.6f}")
    
    # Calculate metrics with optimized weights
    fold_predictions_all = np.array(fold_predictions_all)
    fold_actuals_all = np.array(fold_actuals_all)
    
    mae = np.mean(np.abs(fold_predictions_all - fold_actuals_all))
    rmse = np.sqrt(np.mean((fold_predictions_all - fold_actuals_all) ** 2))
    correlation = np.corrcoef(fold_predictions_all, fold_actuals_all)[0, 1]
    r2 = 1 - (np.sum((fold_actuals_all - fold_predictions_all) ** 2) / 
              np.sum((fold_actuals_all - fold_actuals_all.mean()) ** 2))
    
    # print("\n" + "="*70)
    # print("CROSS-VALIDATION SUMMARY")
    # print("="*70)
    # print(f"Mean Adjusted Sharpe:        {np.mean(scores):.6f} ± {np.std(scores):.6f}")
    # print(f"Best Fold Sharpe:            {np.max(scores):.6f}")
    # print(f"Worst Fold Sharpe:           {np.min(scores):.6f}")
    # print(f"\nPrediction Quality Metrics:")
    # print(f"  MAE (Position Error):      {mae:.6f}")
    # print(f"  RMSE (Position Error):     {rmse:.6f}")
    # print(f"  R² Score:                  {r2:.6f}")
    # print(f"  Correlation (Pred/Actual): {correlation:.6f}")
    # print(f"\nPosition Statistics:")
    # print(f"  Mean Position:             {fold_predictions_all.mean():.4f}")
    # print(f"  Position Std Dev:          {fold_predictions_all.std():.4f}")
    # print(f"  Min Position:              {fold_predictions_all.min():.4f}")
    # print(f"  Max Position:              {fold_predictions_all.max():.4f}")
    
    # Train final models
    print("\n" + "="*70)
    print("TRAINING FINAL MODELS ON FULL DATA")
    print("="*70)
    
    m1 = lgb.LGBMRegressor(**best_params_mom)
    m2 = lgb.LGBMRegressor(**best_params_rev)
    m3 = lgb.LGBMRegressor(**best_params_vol)
    
    m1.fit(X[mom_features], y)
    m2.fit(X[rev_features], y)
    m3.fit(X[vol_features], y)
    
    # Save to /kaggle/working/
    m1.booster_.save_model(MODEL_SAVE_PATH_MOM)
    m2.booster_.save_model(MODEL_SAVE_PATH_REV)
    m3.booster_.save_model(MODEL_SAVE_PATH_VOL)
    print("✓ All models saved successfully to /kaggle/working/!")
    
    # Save metadata for inference
    metadata = {
        'best_weights': best_weights,
        'best_scale': best_scale,
        'dropped_corr': dropped_corr,
        'mom_features': mom_features,
        'rev_features': rev_features,
        'vol_features': vol_features
    }
    np.save(METADATA_SAVE_PATH, metadata)
    print("✓ Metadata saved successfully to /kaggle/working/model_metadata.npy")

    # The rest of the logic for local testing is kept, but it's optional for the Training Notebook
    if not os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
        # Test predictions
        te = feature_engineer(te, is_inference=True)
        X_test = te.drop(columns=["date_id", "forward_returns", "risk_free_rate"], errors='ignore')
        X_test = X_test[[c for c in X_test.columns if c not in dropped_corr]]
        
        pt = np.clip(1 + np.tanh((best_weights[0] * m1.predict(X_test[mom_features]) + 
                                  best_weights[1] * m2.predict(X_test[rev_features]) + 
                                  best_weights[2] * m3.predict(X_test[vol_features])) * best_scale), 0, 2)
        
        # print("\n" + "="*70)
        # print("TEST SET PREDICTIONS")
        # print("="*70)
        # print(f"Predictions generated:         {len(pt)}")
        # print(f"Mean test position:          {pt.mean():.4f}")
        # print(f"Test position std:           {pt.std():.4f}")
        # print(f"Min test position:           {pt.min():.4f}")
        # print(f"Max test position:           {pt.max():.4f}")
        
        sub = pd.DataFrame({"date_id": te["date_id"].astype("int64"), TARGET_COL: pt})
        pq.write_table(
            pa.Table.from_pandas(sub, schema=pa.schema([("date_id", pa.int64()), (TARGET_COL, pa.float64())])),
            "/kaggle/working/submission.parquet"
        )
        
        print("\n✓ Submission file created: /kaggle/working/submission.parquet")
    
    print("="*70 + "\n")
    
    return m1, m2, m3


# PREDICT FUNCTION 
def _safe_load_model():
    """Loads models and metadata from the specified Kaggle Input path."""
    global _initialized, _model_mom, _model_rev, _model_vol, _metadata
    if _initialized:
        return _model_mom, _model_rev, _model_vol, _metadata

    print(f"Loading models from: {MODEL_LOAD_DIR}")
    _model_mom = lgb.Booster(model_file=MODEL_LOAD_PATH_MOM)
    _model_rev = lgb.Booster(model_file=MODEL_LOAD_PATH_REV)
    _model_vol = lgb.Booster(model_file=MODEL_LOAD_PATH_VOL)
    
    _metadata = np.load(METADATA_LOAD_PATH, allow_pickle=True).item()
    
    _initialized = True
    print("Models and metadata loaded successfully.")
    return _model_mom, _model_rev, _model_vol, _metadata

def predict(test):
    """The function used by the inference server to generate a single prediction."""
    df = test.to_pandas()
    # Load models and metadata from the input path
    m1, m2, m3, metadata = _safe_load_model()
    
    # Apply feature engineering
    df = feature_engineer(df, is_inference=True)
    
    # Drop correlated features as recorded in metadata
    df = df[[c for c in df.columns if c not in metadata['dropped_corr']]]
    
    # Select features for each model
    X_m = df[metadata['mom_features']]
    X_r = df[metadata['rev_features']]
    X_v = df[metadata['vol_features']]
    
    # Ensemble prediction
    p = (metadata['best_weights'][0] * m1.predict(X_m) + 
         metadata['best_weights'][1] * m2.predict(X_r) + 
         metadata['best_weights'][2] * m3.predict(X_v))
    
    # Apply scaling and clipping to get the final allocation
    alloc = float(np.clip(1 + np.tanh(p[-1] * metadata['best_scale']), 0, 2))
    return alloc

# ENTRY POINT 
if __name__ == "__main__":
    if not os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
        # Training Notebook (Offline Run)
        print("Running in TRAINING Mode...")
        train_model()
        print("Local run complete. Save this notebook's output as a dataset.")
    else:
        # Submission Notebook (Inference Only Run)
        # This will load the models and metadata from the mounted dataset path (MODEL_LOAD_DIR)
        print("Running in INFERENCE Mode...")
        kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict).serve()